In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

In [3]:
from scripts.downloader import fetch_all_polls

In [4]:
all_polls = fetch_all_polls(cleanup=True, refresh=False)

In [ ]:
all_polls.date_started.map(lambda x: f'{x.year}-{x.month}').value_counts()

In [ ]:
#all_polls = pd.read_csv('poll_database/poll_database.csv')

In [7]:
all_polls.to_csv('poll_database/poll_database.csv', index=False)
all_polls.to_excel('poll_database/poll_database.xlsx', index=False)
all_polls.to_json('poll_database/poll_database.json', index=False, orient='split')

In [8]:
display_columns = ['date_started', 'date_concluded', 'pollster', 'conservative', 'labour', 'liberal_democrat', 'lead']

In [9]:
all_polls['anti_conservative'] = all_polls[
    ['labour', 'liberal_democrat', 'green', 'scottish_national_party', 'plaid_cymru']].fillna(0).sum(axis=1)

In [10]:
all_polls[(all_polls.anti_conservative.fillna(0)) >= 75][
    ['date_concluded', 'pollster', 'labour', 'liberal_democrat', 'green', 'scottish_national_party', 'plaid_cymru',
     'anti_conservative', 'lead']]

,date_concluded,pollster,labour,liberal_democrat,green,scottish_national_party,plaid_cymru,anti_conservative,lead
101,2022-10-22,Omnisis,56.0,10.0,4.0,4.0,1.0,75.0,lab+34.0
103,2022-10-21,YouGov,56.0,10.0,4.0,4.0,1.0,75.0,lab+37.0
105,2022-10-20,PeoplePolling,53.0,11.0,6.0,5.0,NaN,75.0,lab+39.0
108,2022-10-19,Redfield & Wilton Strategies,55.0,12.0,4.0,4.0,NaN,75.0,lab+36.0
111,2022-10-16,Redfield & Wilton Strategies,56.0,11.0,5.0,4.0,NaN,76.0,lab+36.0
4576,1996-01-08,Gallup,60.5,14.5,NaN,NaN,NaN,75.0,lab+39.5
4588,1995-10-30,Gallup,61.0,14.5,NaN,NaN,NaN,75.5,lab+39.5
4622,1995-02-27,Gallup,60.5,15.0,NaN,NaN,NaN,75.5,lab+40.0
4629,1995-01-09,Gallup,62.0,14.0,NaN,NaN,NaN,76.0,lab+43.5
4658,1994-05-30,Gallup,54.0,21.5,NaN,NaN,NaN,75.5,lab+32.5


In [11]:
all_polls.query("conservative>=labour")[display_columns]

,date_started,date_concluded,pollster,conservative,labour,liberal_democrat,lead
309,2022-03-17,2022-03-21,Kantar Public,36.0,36.0,12.0,Tie
404,2021-12-06,2021-12-06,Redfield & Wilton Strategies,38.0,36.0,9.0,con+2.0
405,2021-12-03,2021-12-05,Savanta,38.0,37.0,9.0,con+1.0
407,2021-12-01,2021-12-02,YouGov,36.0,33.0,9.0,con+3.0
409,2021-11-29,2021-12-01,Find Out Now,36.0,35.0,11.0,con+1.0
...,...,...,...,...,...,...,...
6023,1970-12-13,1970-12-13,Gallup,46.0,44.5,NaN,con+1.5
6025,1970-10-18,1970-10-18,Gallup,46.5,46.5,NaN,Tie
6026,1970-09-13,1970-09-13,Gallup,46.5,44.0,NaN,con+2.5
6027,1970-08-16,1970-08-16,Gallup,47.0,43.5,NaN,con+3.5


In [ ]:
all_polls.query("liberal_democrat + 4 > conservative")[display_columns]

In [ ]:
all_polls.query("pollster == 'YouGov' and green<brexit_party")[display_columns + ['green', 'reform_uk', 'brexit_party']]

In [ ]:
all_polls[(all_polls.anti_conservative.fillna(0)) >= 15][
    ['date_concluded', 'pollster', 'labour', 'liberal_democrat', 'green', 'scottish_national_party', 'plaid_cymru',
     'anti_conservative', 'lead']]

In [ ]:
all_polls.query("labour>conservative and lead_value>=35")[display_columns]

In [ ]:
all_polls.query("labour>55")[display_columns]

In [ ]:
all_polls.query("pollster=='Ipsos MORI' and labour>conservative and lead_value>=35")[display_columns]

In [ ]:
all_polls.query("pollster=='ICM Research'")[display_columns]

In [ ]:
all_polls.query("pollster=='Savanta ComRes' and labour >42")[display_columns]

In [ ]:
all_polls.query("pollster=='Savanta ComRes' and lead_value >= 20")[display_columns]

In [ ]:
all_polls.query("labour>53")[display_columns]

In [ ]:
all_polls.query("conservative<29 and labour>conservative and lead_value>14")[display_columns]

In [ ]:
all_polls.loc[(all_polls.labour >= 50), 'maj_lead'] = 1
all_polls['maj_lead'] = all_polls['maj_lead'].fillna(0)
all_polls['maj_lead_in_row'] = 0
for ix, value in all_polls['maj_lead'].iteritems():
    if value >= 1:
        polls_before = all_polls.iloc[:ix]['maj_lead']
        polls_after = all_polls.iloc[ix:]['maj_lead']
        next_untrue_poll = polls_after[polls_after == 0].index[0]
        after_polls = next_untrue_poll - ix

        if len(polls_before[polls_before == 0]) == 0:
            before_polls = len(polls_before)
        else:
            last_untrue_poll = polls_before[polls_before == 0].index[-1]
            before_polls = (ix - last_untrue_poll) - 1

        total_polls = before_polls + after_polls
        all_polls.loc[ix, 'maj_lead_in_row'] = total_polls
    else:
        total_polls = 0

In [ ]:
all_polls[all_polls['maj_lead_in_row'] > 5][
    ['date_concluded', 'pollster', 'conservative', 'labour', 'liberal_democrat', 'lead',
     'maj_lead_in_row']].to_clipboard(sep='\t')

In [ ]:
all_polls

In [ ]:
all_polls.loc[(all_polls.labour > all_polls.conservative) & (all_polls.lead_value >= 10), 'dd_lead'] = 1
all_polls['dd_lead'] = all_polls['dd_lead'].fillna(0)

In [ ]:
all_polls.loc[(all_polls.lead_value >= 10), 'dd_lead'] = 1
all_polls['dd_lead'] = all_polls['dd_lead'].fillna(0)

In [ ]:
all_polls['dd_in_row'] = 0

In [ ]:
for ix, value in all_polls['dd_lead'].iteritems():
    if value >= 1:
        polls_before = all_polls.iloc[:ix]['dd_lead']
        polls_after = all_polls.iloc[ix:]['dd_lead']
        next_untrue_poll = polls_after[polls_after == 0].index[0]
        after_polls = next_untrue_poll - ix

        if len(polls_before[polls_before == 0]) == 0:
            before_polls = len(polls_before)
        else:
            last_untrue_poll = polls_before[polls_before == 0].index[-1]
            before_polls = (ix - last_untrue_poll) - 1

        total_polls = before_polls + after_polls
        all_polls.loc[ix, 'dd_in_row'] = total_polls
    else:
        total_polls = 0



In [ ]:
all_polls[all_polls['dd_in_row'] > 22][
    ['date_concluded', 'pollster', 'conservative', 'labour', 'liberal_democrat', 'lead', 'dd_in_row']]

In [ ]:
polls_after

In [ ]:
before_polls

In [ ]:
polls_after



In [ ]:
polls_before[polls_before == 0]

In [ ]:
all_polls.query("pollster=='Savanta ComRes' and labour>conservative and lead_value>=14")[display_columns]

In [ ]:
all_polls.query("lead_value >= 15 and labour>conservative")[display_columns]

In [ ]:
all_polls.query("lead_value >= 18 and labour>conservative")[display_columns]

In [ ]:
all_polls.query("conservative <29")[display_columns]

In [ ]:
all_polls.query("pollster=='YouGov' and liberal_democrat > 54")[display_columns]

In [ ]:
all_polls.query("pollster=='Ipsos MORI' and liberal_democrat >=15")[display_columns]

In [ ]:
all_polls.query("pollster=='YouGov' and liberal_democrat >12")[display_columns]

In [ ]:
all_polls.query("labour + green >= 50")[display_columns]

In [ ]:
all_polls.query("pollster=='Opinium' and labour >= 41")[display_columns]

In [ ]:
#all_polls.query("pollster == 'YouGov' and conservative <31")[display_columns]

In [ ]:
reporting_date = 'date_started'

In [ ]:
most_recent_date = all_polls[reporting_date].max()
one_year_polls = all_polls[
    all_polls[reporting_date] >= (most_recent_date + relativedelta(months=-17)).replace(day=1)].copy()

In [ ]:
pollsters_latest = pd.DataFrame()
for pollster in one_year_polls.pollster.unique():
    pollster_data = one_year_polls.query(f'pollster == "{pollster}"')
    latest_poll = pollster_data.iloc[0]
    pollsters_latest = pollsters_latest.append(latest_poll)

In [ ]:
top_five = {'Ipsos MORI': 100,
            'Opinium': 79,
            'YouGov': 77,
            'Kantar Public': 69,
            'Number Cruncher Politics': 61,
            'Survation': 59,
            'Panelbase': 57,
            'Deltapoll': 46,
            'BMG Research': 44,
            'FocalData': 39,
            'ICM Research': 35,
            'Savanta': 34,
            'Find Out Now': 34*0.3,  # Ludicrous Local Election Polling
            'Redfield & Wilton Strategies': 34*.75,
            'Techne UK': 34*.75,
            'JL Partners Polls': 34*.75,
            'PeoplePolling': 34*.75,
            'Omnisis': 34*.75,
            }

top_two = {'Ipsos MORI': 100,
           'Survation': 85,
           'Kantar Public': 85,
           'Panelbase': 74,
           'Number Cruncher Politics': 71,
           'Opinium': 71,
           'YouGov': 67,
           'Deltapoll': 50,
           'FocalData': 42,
           'BMG Research': 39,
           'Savanta': 29,
           'ICM Research': 27,
           'Find Out Now': 34*0.3,  # Ludicrous Local Election Polling
           'Redfield & Wilton Strategies': 34*.75,
           'Techne UK': 34*.75,
           'JL Partners Polls': 34*.75,
           'PeoplePolling': 34*.75,
           'Omnisis': 34*.75,
           }

In [ ]:
(71*3 + 79*2 + 100)/6

In [ ]:
recency_weights = {
    -1: 100,
    5: 100,
    14: 70,
    28: 5,
    90: 0
}


def get_recency_weight(poll_date, current_date, return_category=False):
    days_since_poll = (current_date - poll_date).days
    days_start_of_range = weight_start_of_range = 0
    category = 0
    for day, weight in recency_weights.items():
        category += 1
        if days_since_poll > day:
            days_start_of_range = day
            weight_start_of_range = weight
        else:
            percent_through_range = (days_since_poll - days_start_of_range)/(day - days_start_of_range)
            full_weight = weight_start_of_range - ((weight_start_of_range - weight)*percent_through_range)
            if return_category:
                return category - 2
            else:
                return full_weight
    if return_category:
        return category - 1
    else:
        return 0


In [ ]:
def get_pollster_weight(pollster):
    if pollster not in top_two:
        return 0
    top_five_score = top_five[pollster]
    top_two_score = top_two[pollster]
    pollster_w = (3*top_two_score + 2*top_five_score + 100)/6
    return pollster_w


In [ ]:
for pollster in top_two.keys():
    print(f' {pollster:30s}: {get_pollster_weight(pollster):.1f}')

In [ ]:
from scripts.constants import major_parties

In [ ]:
pollsters_latest['recency_weight'] = pollsters_latest[reporting_date].apply(get_recency_weight,
                                                                            current_date=most_recent_date)
pollsters_latest['pollster_weight'] = pollsters_latest['pollster'].apply(get_pollster_weight)
pollsters_latest['poll_weight'] = pollsters_latest['pollster_weight']*(pollsters_latest['recency_weight']/100)

In [ ]:
most_recent_date

In [ ]:
current_average = pd.DataFrame(columns=major_parties + ['lead'], index=['Polling Average'])
lead_party = ''
max_vote_share = 0
second_vote_share = 0
for party in major_parties:
    party_vote_share = pollsters_latest[party].dot(pollsters_latest['poll_weight'])/pollsters_latest[
        'poll_weight'].sum()
    if party_vote_share > max_vote_share:
        second_vote_share = max_vote_share
        max_vote_share = party_vote_share
        lead_party = party
    else:
        if party_vote_share > second_vote_share:
            second_vote_share = party_vote_share
    current_average.loc['Polling Average', party] = party_vote_share
    #current_average.loc['Polling Average', party] = f'{party_vote_share:.1f}%'
#current_average.loc['Polling Average', 'lead'] = f'{lead_party:.3s}+{max_vote_share - second_vote_share:.1f}%'
current_average.loc['Polling Average', 'lead_value'] = max_vote_share - second_vote_share


In [ ]:
one_year_polls['poll_month'] = one_year_polls[reporting_date].apply(lambda x: x.replace(day=1))

In [ ]:
pollster_monthly_summary = one_year_polls.groupby(['poll_month', 'pollster'])[major_parties].agg(['mean', 'count'])
pollster_monthly_summary.columns = [x if i == 0 else f'count{major_parties.index(x)}' for x in major_parties for i in
                                    range(0, 2)]
pollster_monthly_summary.drop(columns=[f'count{major_parties.index(x)}' for x in major_parties][1:], inplace=True)
pollster_monthly_summary.rename(columns={"count0": "count"}, inplace=True)

agg_info = {x: ['mean'] for x in major_parties}
agg_info['count'] = ['count', 'sum']
monthly_summary = pollster_monthly_summary.reset_index().groupby(['poll_month']).agg(agg_info)
monthly_summary.columns = major_parties + ['pollster_count', 'poll_count']

In [ ]:
from scripts.functions import format_lead

monthly_summary[['lead', 'lead_value']] = monthly_summary.apply(format_lead, axis=1, result_type='expand')

In [ ]:
display(pollsters_latest.sort_values(by='poll_weight', ascending=False)[display_columns])
display(current_average)
display(monthly_summary.iloc[:0:-1])

In [ ]:
from html2image import Html2Image

hti = Html2Image()

In [ ]:



def result_to_opacity(result, range_min, range_max):
    result_range = range_max - range_min
    result_percent = (result - range_min)/result_range
    result_256 = int(result_percent*255)
    return f'{result_256:02x}'


def add_background_colour_to_cells(df, lead_only=False):
    from scripts.constants import party_colors, major_parties
    background_df = df.copy()
    max_party_lead = background_df['lead_value'].max()

    for party in major_parties:
        color = party_colors[party]
        party_max = background_df[party].max()
        party_min = background_df[party].min() - 0.5
        if not lead_only:
            background_df[party] = background_df[party].apply(
                lambda x: f'background-color: {color}{result_to_opacity(x, party_min, party_max)}'
            )
        else:
            background_df[party] = ''
        party_in_lead_mask = df[party] == df[major_parties].max(axis=1)
        background_df.loc[party_in_lead_mask, 'lead_value'] = df['lead_value'].apply(
            lambda x: f'background-color: {color}{result_to_opacity(x, 0, max_party_lead)}'
        )

    if reporting_date in background_df.columns:
        background_df[reporting_date] = background_df[reporting_date].apply(get_recency_weight,
                                                                            current_date=most_recent_date)
        background_df[reporting_date] = background_df[reporting_date].apply(
            lambda x: f'background-color: #A9D08E{result_to_opacity(x, 0, 100)}'
        )

    for column in background_df.columns:
        if not (column in major_parties or column in ['lead_value', reporting_date]):
            background_df[column] = ''
    return background_df

In [ ]:
def polls_to_html(dataframe, title, highlight_party_columns=True, precision=0):
    dataframe.index.name = 'id'
    table_id = '_'.join(title.split(' ')).lower()

    from pandas.io.formats.style import Styler
    #styler = dataframe.style
    styler = Styler(dataframe, uuid=table_id, cell_ids=True)

    styler.apply(add_background_colour_to_cells, axis=None,
                 lead_only=not (highlight_party_columns))

    styler.format({
        "pollster": lambda x: f"{x}",
        "pollster_count": lambda x: f"{x}",
        "poll_count": lambda x: f"{x}",
        "conservative": lambda x: f"{x:.{precision}f}%",
        "labour": lambda x: f"{x:.{precision}f}%",
        "liberal_democrat": lambda x: f"{x:.{precision}f}%",
        "lead_value": lambda x: f"{x:.{precision}f}%",
        reporting_date: lambda x: f"{x:%d-%b}",
        'poll_month': lambda x: f"{x:%b-%y}",
        'poll_weight': lambda x: f"{x:.0f}",
    })

    styler.set_table_attributes(f'id="{table_id}"')
    styler.hide_index()

    # classes = pd.DataFrame([['_'.join(x.split(' ')).lower() for x in dataframe.columns]]
    #                        , index=dataframe.index, columns=dataframe.columns)
    #
    # styler.set_td_classes(classes)

    #styler.set_caption(title)
    styler.set_properties(**{'min-width': '10px', 'font-size': '12pt', 'text-align': 'center'})

    caption_format = {'selector': 'caption',
                      'props': [('text-align', 'center'), ('font-size', '20pt'), ('color', 'black'),
                                ('padding-bottom', '15')]}
    all_cells = {'selector': '', 'props': [('margin', '0 auto'), ('width', '100%')]}
    date_col = {'selector': '.col0', 'props': [('text-align', 'left')]}

    styles = [all_cells, date_col, caption_format]

    if 'poll_weight' in dataframe.columns:
        styles.append({'selector': '.col2', 'props': [('color', '#AAA')]})

    df_as_html = styler.set_table_styles(styles).render()

    for col in range(0, len(dataframe.columns)):
        df_as_html = df_as_html.replace(f'col{col}', dataframe.columns[col])

    df_as_html = (df_as_html
                  .replace('>poll_count', '>Polls')
                  .replace('>pollster_count', '>Pollsters')
                  .replace('>conservative', '>Conservatives')
                  .replace('>labour', '>Labour')
                  .replace('>liberal_democrat', '>Lib Dem')
                  .replace('>lead_value', '>Lead')
                  .replace(f'>{reporting_date}', '>Date')
                  .replace('>poll_month', '>Month')
                  .replace('>pollster', '>Pollster')
                  .replace('>poll_weight', '>w')
                  )
    return df_as_html

In [ ]:
display_columns = [reporting_date, 'pollster', 'conservative', 'labour', 'liberal_democrat', 'lead_value']
top_25_html = polls_to_html(all_polls[display_columns][0:25], title='Last 25 Polls')

In [ ]:
display_columns = ['pollster', reporting_date, 'poll_weight', 'conservative', 'labour', 'liberal_democrat',
                   'lead_value']
df = pollsters_latest.sort_values(by='poll_weight', ascending=False)[display_columns]
pollsters_recent = polls_to_html(df, title='Latest Polls', highlight_party_columns=False)

In [ ]:
display_columns = ['poll_month', 'pollster_count', 'poll_count', 'conservative', 'labour', 'liberal_democrat',
                   'lead_value']
df = monthly_summary.reset_index()[display_columns].iloc[:0:-1]
monthly_averages = polls_to_html(df, title='Monthly Poll Average', highlight_party_columns=True, precision=1)

In [ ]:
polling_average = polls_to_html(current_average[['conservative', 'labour', 'liberal_democrat', 'lead_value']],
                                title='Polling Average', highlight_party_columns=False, precision=1)

In [ ]:

css = "body {background: white;}"
# hti.screenshot(html_str=top_25_html, css_str=css, save_as='tmp.png', size=(800, 650))
# hti.screenshot(html_str=pollsters_recent, css_str=css, save_as='tmp.png', size=(700, 400))

In [ ]:
with open('html_outputs/top_25.html', 'w') as f:
    f.write(top_25_html)
with open('html_outputs/pollsters_recent.html', 'w') as f:
    f.write(pollsters_recent)
with open('html_outputs/monthly_averages.html', 'w') as f:
    f.write(monthly_averages)
with open('html_outputs/polling_average.html', 'w') as f:
    f.write(polling_average)

In [ ]:
from scripts.constants import party_colors, major_parties
import matplotlib.pyplot as plt
from numpy import arange

fig = plt.figure(figsize=(15, 3))
fig.set_facecolor('white')
ax = plt.gca()

# monthly_summary[major_parties[:2]][1:].plot(
#     ax=ax,
#     ylim=[30, 45],
#     color=party_colors,
#     linewidth=3
# )
for party in major_parties[:2]:
    plt.plot_date(
        x=monthly_summary.index[1:],
        y=monthly_summary[party][1:],
        fmt='-',
        color=party_colors[party],
        linewidth=4)

    ax.fill_between(monthly_summary.index[1:], monthly_summary[party][1:],
                    monthly_summary[major_parties[:2]].min(axis=1)[1:], color=party_colors[party], alpha=0.2)
ax.set_yticks(arange(25, 51, 5), minor=False)
major_locator = plt.matplotlib.dates.DayLocator([1])
major_fmt = plt.matplotlib.dates.DateFormatter('%b-%y')

ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_fmt)

ax.yaxis.set_major_formatter(plt.matplotlib.ticker.FuncFormatter(lambda x, p: f'{x/100:.0%}'))

plot_title = 'Opinion Polling Trend'
plt.title(plot_title)

source = 'Source: Wikipedia - Opinion polling for the next United Kingdom general election'
plt.annotate(source, (1, 0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top', ha='right')
fig.savefig('html_outputs/monthly_trend.png', bbox_inches='tight', pad_inches=0.2)

In [ ]:
monthly_summary

In [ ]:
import datetime

In [ ]:
all_polls['anti-Conservative'] = all_polls[
    ['labour', 'liberal_democrat', 'alliance', 'change_uk', 'green', 'liberal', 'plaid_cymru',
     'scottish_national_party', 'social_democratic_party']].sum(axis=1)
all_polls['Conservative+'] = all_polls[
    ['conservative', 'brexit_party', 'reform_uk', 'united_kingdom_independence_party', 'reclaim_party']].sum(axis=1)

(all_polls
 .set_index(reporting_date)
 .rolling(window=15).mean()
 .loc[:datetime.date(2010, 1, 13)]
[['anti-Conservative', 'Conservative+']]
 .plot(
    color=['r', 'b']
    , figsize=[20, 5]
    , grid='y'
    , ylim=[0, 100]
)
)

In [ ]:
group_colours = {'conservative+': '#0087DC',
                 'labour+': '#DC241F',
                 }
groups = list(group_colours.keys())

# group_parties = {'conservative+': ['conservative', 'brexit_party', 'reform_uk','united_kingdom_independence_party', 'reclaim_party'],
#  'anti-Conservative': ['labour', 'liberal_democrat', 'alliance', 'change_uk', 'green', 'liberal', 'plaid_cymru', 'scottish_national_party', 'social_democratic_party'],
#  }

group_parties = {'conservative+': ['conservative', 'brexit_party', 'reform_uk', 'united_kingdom_independence_party',
                                   'reclaim_party'],
                 'labour+': ['labour'],
                 }

two_group_polls = all_polls[all_polls[reporting_date] >= datetime.datetime(1990, 1, 1)].copy()

for group, parties in group_parties.items():
    two_group_polls[group] = two_group_polls[parties].sum(axis=1)
    two_group_polls.drop(
        columns=parties,
        inplace=True,
    )

two_group_polls['poll_month'] = two_group_polls[reporting_date].apply(lambda x: x.replace(day=1))
two_group_polls = two_group_polls[[reporting_date, 'pollster', 'poll_month'] + groups]

two_group_monthly = two_group_polls.groupby(['poll_month', 'pollster'])[groups].agg(['mean', 'count'])
two_group_monthly.columns = [x if i == 0 else f'count{groups.index(x)}' for x in groups for i in range(0, 2)]
two_group_monthly.drop(columns=[f'count{groups.index(x)}' for x in groups][1:], inplace=True)
two_group_monthly.rename(columns={"count0": "count"}, inplace=True)

agg_info = {x: ['mean'] for x in groups}
agg_info['count'] = ['count', 'sum']
two_group_monthly_summary = two_group_monthly.reset_index().groupby(['poll_month']).agg(agg_info)
two_group_monthly_summary.columns = groups + ['pollster_count', 'poll_count']

import matplotlib.pyplot as plt
from numpy import arange

fig = plt.figure(figsize=(15, 5))
fig.set_facecolor('white')
ax = plt.gca()

for group in groups:
    plt.plot_date(
        x=two_group_monthly_summary.index[1:],
        y=two_group_monthly_summary[group][1:],
        fmt='-',
        color=group_colours[group],
        linewidth=4)

    ax.fill_between(two_group_monthly_summary.index[1:], two_group_monthly_summary[group][1:],
                    two_group_monthly_summary[groups[:2]].min(axis=1)[1:], color=group_colours[group], alpha=0.2)
ax.set_yticks(arange(20, 56, 5), minor=False)
# major_locator = plt.matplotlib.dates.DayLocator([1])
# major_fmt = plt.matplotlib.dates.DateFormatter('%b-%y')

major_locator = plt.matplotlib.dates.MonthLocator([1])
major_fmt = plt.matplotlib.dates.DateFormatter("%y")

ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_fmt)

ax.set_xlim([two_group_monthly_summary.index[1], two_group_monthly_summary.index[-1]])

ax.yaxis.set_major_formatter(plt.matplotlib.ticker.FuncFormatter(lambda x, p: f'{x/100:.0%}'))

titleOffset = 0.125
renderer = fig.canvas.get_renderer()
figwidth = ax.get_window_extent().width/0.8

group_parts = [[f"{group}", group_colours[group], 'bold'] for group in groups]
titleparts = [['Opinion Polling Trend', 'k', 'book']] + [a for b in [[x, [" vs ", 'k', 'regular']] for x in group_parts]
                                                         for a in b][:-1]

for part in titleparts:
    txt = fig.text(titleOffset, 0.9, part[0], ha="left", va="bottom", size="large", color=part[1], weight=part[2])
    titleOffset += (txt.get_window_extent(renderer=renderer).width/figwidth)

# plot_title = 'Opinion Polling Trend'
# plt.title(plot_title)

source = 'Source: Wikipedia - Opinion polling for the {year} United Kingdom general election'
plt.annotate(source, (1, 0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top', ha='right')

titleOffset = 0.15
for group, parties in group_parties.items():
    colour = group_colours[group]
    fig.text(0.35, titleOffset, group, ha="left", va="bottom", size="medium", color=colour, weight='bold')
    fig.text(0.445, titleOffset, ': ' + ', '.join(parties).replace('_', ' '), ha="left", va="bottom", size="small",
             color='k', weight='regular')
    titleOffset += 0.05
#fig.savefig('html_outputs/monthly_trend.png', bbox_inches = 'tight', pad_inches = 0.2)

In [ ]:
group_colours = {'conservative+': '#0087DC',
                 'anti-Conservative': '#DC241F',
                 }
groups = list(group_colours.keys())

# group_parties = {'conservative+': ['conservative', 'brexit_party', 'reform_uk','united_kingdom_independence_party', 'reclaim_party'],
#  'anti-Conservative': ['labour', 'liberal_democrat', 'alliance', 'change_uk', 'green', 'liberal', 'plaid_cymru', 'scottish_national_party', 'social_democratic_party'],
#  }

group_parties = {'conservative+': ['conservative', 'brexit_party', 'reform_uk', 'united_kingdom_independence_party',
                                   'reclaim_party'],
                 'anti-Conservative': ['labour', 'change_uk', 'green', 'plaid_cymru', 'scottish_national_party',
                                       'liberal_democrat'],
                 }

two_group_polls = all_polls[all_polls[reporting_date] >= datetime.datetime(1990, 1, 1)].copy()

for group, parties in group_parties.items():
    two_group_polls[group] = two_group_polls[parties].sum(axis=1)
    two_group_polls.drop(
        columns=parties,
        inplace=True,
    )

two_group_polls['poll_month'] = two_group_polls[reporting_date].apply(lambda x: x.replace(day=1))
two_group_polls = two_group_polls[[reporting_date, 'pollster', 'poll_month'] + groups]

two_group_monthly = two_group_polls.groupby(['poll_month', 'pollster'])[groups].agg(['mean', 'count'])
two_group_monthly.columns = [x if i == 0 else f'count{groups.index(x)}' for x in groups for i in range(0, 2)]
two_group_monthly.drop(columns=[f'count{groups.index(x)}' for x in groups][1:], inplace=True)
two_group_monthly.rename(columns={"count0": "count"}, inplace=True)

agg_info = {x: ['mean'] for x in groups}
agg_info['count'] = ['count', 'sum']
two_group_monthly_summary = two_group_monthly.reset_index().groupby(['poll_month']).agg(agg_info)
two_group_monthly_summary.columns = groups + ['pollster_count', 'poll_count']

import matplotlib.pyplot as plt
from numpy import arange

fig = plt.figure(figsize=(15, 5))
fig.set_facecolor('white')
ax = plt.gca()

for group in groups:
    plt.plot_date(
        x=two_group_monthly_summary.index[1:],
        y=two_group_monthly_summary[group][1:],
        fmt='-',
        color=group_colours[group],
        linewidth=4)

    ax.fill_between(two_group_monthly_summary.index[1:], two_group_monthly_summary[group][1:],
                    two_group_monthly_summary[groups[:2]].min(axis=1)[1:], color=group_colours[group], alpha=0.2)
ax.set_yticks(arange(20, 81, 5), minor=False)
# major_locator = plt.matplotlib.dates.DayLocator([1])
# major_fmt = plt.matplotlib.dates.DateFormatter('%b-%y')

major_locator = plt.matplotlib.dates.MonthLocator([1])
major_fmt = plt.matplotlib.dates.DateFormatter("%y")

ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_fmt)

ax.set_xlim([two_group_monthly_summary.index[1], two_group_monthly_summary.index[-1]])

ax.yaxis.set_major_formatter(plt.matplotlib.ticker.FuncFormatter(lambda x, p: f'{x/100:.0%}'))

titleOffset = 0.125
renderer = fig.canvas.get_renderer()
figwidth = ax.get_window_extent().width/0.8

group_parts = [[f"{group}", group_colours[group], 'bold'] for group in groups]
titleparts = [['Opinion Polling Trend', 'k', 'book']] + [a for b in [[x, [" vs ", 'k', 'regular']] for x in group_parts]
                                                         for a in b][:-1]

for part in titleparts:
    txt = fig.text(titleOffset, 0.9, part[0], ha="left", va="bottom", size="large", color=part[1], weight=part[2])
    titleOffset += (txt.get_window_extent(renderer=renderer).width/figwidth)

# plot_title = 'Opinion Polling Trend'
# plt.title(plot_title)

source = 'Source: Wikipedia - Opinion polling for the {year} United Kingdom general election'
plt.annotate(source, (1, 0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top', ha='right')

titleOffset = 0.15
for group, parties in group_parties.items():
    colour = group_colours[group]
    fig.text(0.35, titleOffset, group, ha="left", va="bottom", size="medium", color=colour, weight='bold')
    fig.text(0.445, titleOffset, ': ' + ', '.join(parties).replace('_', ' '), ha="left", va="bottom", size="small",
             color='k', weight='regular')
    titleOffset += 0.05
#fig.savefig('html_outputs/monthly_trend.png', bbox_inches = 'tight', pad_inches = 0.2)

In [ ]:
group_colours = {'conservative+': '#0087DC',
                 'anti-Conservative': '#DC241F',
                 }
groups = list(group_colours.keys())

# group_parties = {'conservative+': ['conservative', 'brexit_party', 'reform_uk','united_kingdom_independence_party', 'reclaim_party'],
#  'anti-Conservative': ['labour', 'liberal_democrat', 'alliance', 'change_uk', 'green', 'liberal', 'plaid_cymru', 'scottish_national_party', 'social_democratic_party'],
#  }

group_parties = {'conservative+': ['conservative', 'brexit_party', 'reform_uk', 'united_kingdom_independence_party',
                                   'reclaim_party'],
                 'anti-Conservative': ['labour', 'change_uk', 'green', 'plaid_cymru', 'scottish_national_party',
                                       'liberal_democrat'],
                 }

two_group_polls = all_polls[all_polls[reporting_date] >= datetime.datetime(2016, 1, 1)].copy()

for group, parties in group_parties.items():
    two_group_polls[group] = two_group_polls[parties].sum(axis=1)
    two_group_polls.drop(
        columns=parties,
        inplace=True,
    )

two_group_polls['poll_month'] = two_group_polls[reporting_date].apply(lambda x: x.replace(day=1))
two_group_polls = two_group_polls[[reporting_date, 'pollster', 'poll_month'] + groups]

two_group_monthly = two_group_polls.groupby(['poll_month', 'pollster'])[groups].agg(['mean', 'count'])
two_group_monthly.columns = [x if i == 0 else f'count{groups.index(x)}' for x in groups for i in range(0, 2)]
two_group_monthly.drop(columns=[f'count{groups.index(x)}' for x in groups][1:], inplace=True)
two_group_monthly.rename(columns={"count0": "count"}, inplace=True)

agg_info = {x: ['mean'] for x in groups}
agg_info['count'] = ['count', 'sum']
two_group_monthly_summary = two_group_monthly.reset_index().groupby(['poll_month']).agg(agg_info)
two_group_monthly_summary.columns = groups + ['pollster_count', 'poll_count']

import matplotlib.pyplot as plt
from numpy import arange

fig = plt.figure(figsize=(15, 5))
fig.set_facecolor('white')
ax = plt.gca()

for group in groups:
    plt.plot_date(
        x=two_group_monthly_summary.index[1:],
        y=two_group_monthly_summary[group][1:],
        fmt='-',
        color=group_colours[group],
        linewidth=4)

    ax.fill_between(two_group_monthly_summary.index[1:], two_group_monthly_summary[group][1:],
                    two_group_monthly_summary[groups[:2]].min(axis=1)[1:], color=group_colours[group], alpha=0.2)
ax.set_yticks(arange(20, 81, 5), minor=False)
# major_locator = plt.matplotlib.dates.DayLocator([1])
# major_fmt = plt.matplotlib.dates.DateFormatter('%b-%y')

major_locator = plt.matplotlib.dates.MonthLocator([1])
major_fmt = plt.matplotlib.dates.DateFormatter("%Y")

ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_fmt)

ax.set_xlim([two_group_monthly_summary.index[1], two_group_monthly_summary.index[-1]])

ax.yaxis.set_major_formatter(plt.matplotlib.ticker.FuncFormatter(lambda x, p: f'{x/100:.0%}'))

titleOffset = 0.125
renderer = fig.canvas.get_renderer()
figwidth = ax.get_window_extent().width/0.8

group_parts = [[f"{group}", group_colours[group], 'bold'] for group in groups]
titleparts = [['Opinion Polling Trend', 'k', 'book']] + [a for b in [[x, [" vs ", 'k', 'regular']] for x in group_parts]
                                                         for a in b][:-1]

for part in titleparts:
    txt = fig.text(titleOffset, 0.9, part[0], ha="left", va="bottom", size="large", color=part[1], weight=part[2])
    titleOffset += (txt.get_window_extent(renderer=renderer).width/figwidth)

# plot_title = 'Opinion Polling Trend'
# plt.title(plot_title)

source = 'Source: Wikipedia - Opinion polling for the {year} United Kingdom general election'
plt.annotate(source, (1, 0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top', ha='right')

titleOffset = 0.15
for group, parties in group_parties.items():
    colour = group_colours[group]
    fig.text(0.35, titleOffset, group, ha="left", va="bottom", size="medium", color=colour, weight='bold')
    fig.text(0.445, titleOffset, ': ' + ', '.join(parties).replace('_', ' '), ha="left", va="bottom", size="small",
             color='k', weight='regular')
    titleOffset += 0.05
#fig.savefig('html_outputs/monthly_trend.png', bbox_inches = 'tight', pad_inches = 0.2)

In [ ]:
one_year_polls['poll_month'] = one_year_polls[reporting_date].apply(lambda x: x.replace(day=1))

In [ ]:
kantar_polls = all_polls.query('pollster == "Kantar Public" and date_concluded> "2019-01-01"').copy()
other_polls = all_polls.query('pollster != "Kantar Public" and date_concluded> "2019-01-01"').copy()

In [ ]:
kantar_polls['poll_month'] = kantar_polls[reporting_date].apply(lambda x: x.replace(day=1))
other_polls['poll_month'] = other_polls[reporting_date].apply(lambda x: x.replace(day=1))

In [ ]:
other_pollster_monthly_summary = other_polls.groupby(['poll_month', 'pollster'])[major_parties].agg(['mean', 'count'])
other_pollster_monthly_summary.columns = [x if i == 0 else f'count{major_parties.index(x)}' for x in major_parties for i
                                          in
                                          range(0, 2)]
other_pollster_monthly_summary.drop(columns=[f'count{major_parties.index(x)}' for x in major_parties][1:], inplace=True)
other_pollster_monthly_summary.rename(columns={"count0": "count"}, inplace=True)

agg_info = {x: ['mean'] for x in major_parties}
agg_info['count'] = ['count', 'sum']
other_monthly_summary = other_pollster_monthly_summary.reset_index().groupby(['poll_month']).agg(agg_info)
other_monthly_summary.columns = major_parties + ['pollster_count', 'poll_count']

In [ ]:
kantar_pollster_monthly_summary = kantar_polls.groupby(['poll_month', 'pollster'])[major_parties].agg(['mean', 'count'])
kantar_pollster_monthly_summary.columns = [x if i == 0 else f'count{major_parties.index(x)}' for x in major_parties for
                                           i in
                                           range(0, 2)]
kantar_pollster_monthly_summary.drop(columns=[f'count{major_parties.index(x)}' for x in major_parties][1:],
                                     inplace=True)
kantar_pollster_monthly_summary.rename(columns={"count0": "count"}, inplace=True)

agg_info = {x: ['mean'] for x in major_parties}
agg_info['count'] = ['count', 'sum']
kantar_monthly_summary = kantar_pollster_monthly_summary.reset_index().groupby(['poll_month']).agg(agg_info)
kantar_monthly_summary.columns = major_parties + ['pollster_count', 'poll_count']

In [ ]:
other_monthly_summary[['kantar_con', 'kantar_lab']] = kantar_monthly_summary[['conservative', 'labour']]

In [ ]:
other_monthly_summary.to_clipboard(sep='\t')

In [ ]:
general_elections = all_polls.query('pollster=="General Election"')
general_election_dates = list(general_elections.date_concluded.unique())
#general_elections = general_elections[general_elections.date_concluded > datetime.datetime(2016,1,1)]

In [ ]:
for ge in general_election_dates[::-1]:
    if ge < np.datetime64('2016-01-01'):
        continue

    print(ge)

In [ ]:
general_elections[general_elections.date_concluded == ge]

In [ ]:
all_polls.query('liberal_democrat + 10 > conservative')

In [ ]:
all_polls.query('labour>56')

In [ ]:
GEs_to_use = [

]

In [ ]:
all_polls.query('pollster=="Techne UK"')

In [ ]:
break

In [ ]:
import pandas as pd
import requests
from io import StringIO

url = 'https://www.markpack.org.uk/files/2022/07/PollBase-Q2-2022.xlsx'
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
req = requests.get(url, headers=headers)



In [ ]:
data = StringIO(req.content)

In [ ]:
pollbase = pd.read_excel(req.content, sheet_name=1)

In [ ]:
pollbase

In [ ]:
all_polls['poll_month'] = all_polls['date_concluded'].apply(lambda x: x.replace(day=1))

In [ ]:
pollster_monthly_summary = all_polls.groupby(['poll_month', 'pollster'])[major_parties].agg(['mean', 'count'])
pollster_monthly_summary.columns = [x if i == 0 else f'count{major_parties.index(x)}' for x in major_parties for i in
                                    range(0, 2)]
pollster_monthly_summary.drop(columns=[f'count{major_parties.index(x)}' for x in major_parties][1:], inplace=True)
pollster_monthly_summary.rename(columns={"count0": "count"}, inplace=True)

agg_info = {x: ['mean'] for x in major_parties}
agg_info['count'] = ['count', 'sum']
monthly_summary = pollster_monthly_summary.reset_index().groupby(['poll_month']).agg(agg_info)
monthly_summary.columns = major_parties + ['pollster_count', 'poll_count']

In [ ]:
monthly_summary.query('conservative<31.5')

In [ ]:
monthly_summary.query('(labour-conservative)>9.3')